In [1]:
from massspecgym.data.datasets import MSnRetrievalDataset
from massspecgym.data.transforms import MolFingerprinter, SpecTokenizer
from massspecgym.data import RetrievalDataset, MassSpecDataModule
import torch

In [2]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_mass.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [4]:
pth_massspecgym_original = "/Users/macbook/CODE/Majer:MassSpecGym/data/MassSpecGym/MassSpecGym.tsv"

In [5]:
# Init hyperparameters
n_peaks = 60
fp_size = 2048
batch_size = 12

# Load dataset
dataset_original = RetrievalDataset(
    pth=pth_massspecgym_original,
    candidates_pth=file_json,
    spec_transform=SpecTokenizer(n_peaks=n_peaks),
    mol_transform=MolFingerprinter(fp_size=fp_size),
)


In [6]:
# Init data module
data_module_original = MassSpecDataModule(
    dataset=dataset_original,
    batch_size=batch_size,
    num_workers=0,
)

In [7]:
data_module_original.prepare_data()
data_module_original.setup()

train_loader_original = data_module_original.train_dataloader()

Train dataset size: 194119
Val dataset size: 19429


In [8]:
tmp_original = []
for batch in train_loader_original:
    print(batch)
    tmp_original = batch
    break

[PROFILE] __getitem__ took 0.001076 seconds
I am not here
{'spec': tensor([[[7.8055e+02, 1.1000e+00],
         [9.8984e+01, 5.9727e-03],
         [1.0411e+02, 7.6491e-03],
         ...,
         [5.9748e+02, 1.0840e-02],
         [7.2147e+02, 2.0949e-03],
         [7.8055e+02, 2.3680e-03]],

        [[3.6817e+02, 1.1000e+00],
         [7.9054e+01, 5.3699e-02],
         [8.0049e+01, 1.0696e-01],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[3.8718e+02, 1.1000e+00],
         [8.5028e+01, 6.7860e-02],
         [1.0304e+02, 1.6163e-01],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        ...,

        [[3.3906e+02, 1.1000e+00],
         [1.1104e+02, 5.2072e-02],
         [2.1501e+02, 1.3183e-02],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[1.920

In [9]:
# Check the keys in the batch
print(tmp_original.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_original:
    print("batch_ptr is present:", tmp_original['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256])


In [10]:
tmp_original.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [11]:
tmp_original['spec'].shape, tmp_original['mol'].shape, 

(torch.Size([12, 61, 2]), torch.Size([12, 2048]))

In [12]:
tmp_original['precursor_mz']

tensor([780.5540, 368.1704, 387.1802, 271.1300, 439.2930, 499.1770, 437.1210,
        257.0820, 801.4760, 339.0634, 192.0660, 269.1051])

In [13]:
tmp_original['adduct'], 

(['[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+Na]+',
  '[M+Na]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+'],)

In [14]:
tmp_original['mol_freq']

tensor([ 52., 101.,   3.,   8.,   3.,   5.,  52.,  78.,   9.,   3.,  31.,  46.])

In [15]:
tmp_original['labels']

tensor([ True, False, False,  ..., False, False, False])

In [16]:
sum(tmp_original['labels'])

tensor(12)

In [17]:
torch.nonzero(tmp_original['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768, 1024, 1280, 1536, 1792, 2048, 2304, 2560, 2816])

In [18]:
len(tmp_original['labels'])

3072

In [19]:
tmp_original['batch_ptr']

tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256])

In [20]:
sum(tmp_original['batch_ptr'])

tensor(3072)

In [21]:
len(tmp_original['candidates'])

3072

In [22]:
tmp_original['candidates'][5].shape

torch.Size([2048])

In [23]:
len(tmp_original['candidates_smiles'])

3072

# MSnRetrieval

In [24]:
from massspecgym.featurize import SpectrumFeaturizer

In [68]:
config = {
    'features': ['collision_energy', 'ionmode', 'adduct', 'spectrum_stats', 'atom_counts', 'value', "retention_time", 'ion_source', 'binned_peaks'],
    'feature_attributes': {
        'atom_counts': {
            'top_n_atoms': 12,
            'include_other': True,
        },
    },
}

In [69]:
featurizer = SpectrumFeaturizer(config, mode='torch')

In [70]:
mass_spectra = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
retrieval_candidates = "/Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/MassSpecGymMSn_retrieval_candidates_mass.json"
retrieval_candidates_cached = "/Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/MassSpecGymMSn_retrieval_candidates_mass.h5"

In [71]:
# file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/min_sample_trees.mgf"


In [72]:
# Init hyperparameters
fingerprint_size = 2048
batch_size = 12

# Load dataset
dataset_msn = MSnRetrievalDataset(
    pth=mass_spectra, # MGF file containing mass spectra
    candidates_pth=retrieval_candidates, # *_mass for standard or *_formula for bonus challenge
    featurizer=featurizer, # Preprocessing of mass spectra
    cache_pth=retrieval_candidates_cached, # Optional, if not provided, it will be created automatically
    mol_transform=MolFingerprinter(fp_size=fingerprint_size), # Cached molecules based on mol_transform
)


Total valid indices: 16476
MSnRetrievalDataset length: 16476
Precomputing candidate-side transformations using multiprocessing with chunking and writing to HDF5...
Using 8 worker processes for precomputation.


Processing chunks: 100%|██████████| 33/33 [15:30<00:00, 28.21s/it]  


Precomputation complete. Cache saved to /Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/MassSpecGymMSn_retrieval_candidates_mass.h5


In [74]:
# Init data module
data_module_msn = MassSpecDataModule(
    dataset=dataset_msn,
    batch_size=batch_size, 
    split_pth=split_file, # File defining which molecules belong to which train/val/test split
    num_workers=0,
)

In [75]:
data_module_msn.prepare_data()
data_module_msn.setup()

train_loader_msn = data_module_msn.train_dataloader()

Train dataset size: 12536
Val dataset size: 1952


In [76]:
tmp_msn = []
for batch in train_loader_msn:
    print(batch)
    tmp_msn = batch
    break

{'spec': DataBatch(x=[101, 1039], edge_index=[2, 178], batch=[101], ptr=[13]), 'mol': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'precursor_mz': tensor([ 381.2397,  441.1673,  460.2406,  358.1178,  434.2537,  446.2099,
         363.1308,  428.2557,  384.1199,  352.1512,  501.2721, 1107.5582]), 'adduct': ['[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+NH4]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+'], 'identifier': ['0043815_0000000', '0099370_0000000', '0051774_0000000', '0040597_0000000', '0009435_0000000', '0066248_0000000', '0019564_0000000', '0050992_0000000', '0024453_0000000', '0019706_0000000', '0073358_0000000', '0006442_0000000'], 'mol_freq': tensor([2., 1., 1., 1., 4., 1., 1., 1., 1., 1., 2., 1.]), 'smiles': ['CCC1=C2N=C(C=C(N2N=C1)NCC3=CN=CC=

In [77]:
# Check the keys in the batch
print(tmp_msn.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_msn:
    print("batch_ptr is present:", tmp_msn['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256],
       dtype=torch.int32)


In [78]:
def print_key_types(obj, obj_name):
    print(f"Data types in {obj_name}:")
    for key, value in obj.items():
        print(f"  {key}: {type(value)}")

# Print data types for both objects
print_key_types(tmp_original, "tmp_original")
print_key_types(tmp_msn, "tmp_msn")

Data types in tmp_original:
  spec: <class 'torch.Tensor'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  identifier: <class 'list'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>
Data types in tmp_msn:
  spec: <class 'abc.DataBatch'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  identifier: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>


#### Shold be dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [79]:
tmp_msn.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [80]:
tmp_msn['spec'] 

DataBatch(x=[101, 1039], edge_index=[2, 178], batch=[101], ptr=[13])

In [81]:
tmp_msn['precursor_mz']

tensor([ 381.2397,  441.1673,  460.2406,  358.1178,  434.2537,  446.2099,
         363.1308,  428.2557,  384.1199,  352.1512,  501.2721, 1107.5582])

In [82]:
tmp_msn['mol_freq']

tensor([2., 1., 1., 1., 4., 1., 1., 1., 1., 1., 2., 1.])

In [83]:
tmp_msn['identifier']

['0043815_0000000',
 '0099370_0000000',
 '0051774_0000000',
 '0040597_0000000',
 '0009435_0000000',
 '0066248_0000000',
 '0019564_0000000',
 '0050992_0000000',
 '0024453_0000000',
 '0019706_0000000',
 '0073358_0000000',
 '0006442_0000000']

In [84]:
tmp_msn['adduct']

['[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+NH4]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+']

In [85]:
tmp_msn['mol'].shape

torch.Size([12, 2048])

In [86]:
tmp_msn['smiles']

['CCC1=C2N=C(C=C(N2N=C1)NCC3=CN=CC=C3)N4CCCC[C@H]4CCO',
 'C[C@]12C[C@@H]([C@H]3[C@H]([C@@H]1CC[C@@]2(C(=O)COP(=O)(O)O)O)CCC4=CC(=O)C=C[C@]34C)O',
 'CCC(C)[C@@H](C(=O)OC)NC(=O)N1CCN(CC1)C(C2=CC=C(C=C2)F)C3=CC=C(C=C3)F',
 'C1CN(CCN1C2=CC3=NN=CN3N=C2)C(=O)NC4=CC=CC=C4Cl',
 'C[C@@H]1CC2=CC(=C(C(=C2C3=C(C(=C(C=C3C[C@@H]1C)OC)OC)OC)OC)OC)OC',
 'CC1=CC(=C(C=C1N2C=C(N=C2)C3CC3)C(=O)NC4=CC=CC(=N4)C5=NN=CN5C(C)C)F',
 'C1CCN(CC1)C(=O)CSC2=NC(=C3C4=C(CCCC4)SC3=N2)N',
 'CC1=C(C=C(C=C1)NC2=C3C=NN(C3=NC(=N2)N4CCN(CC4)CC5=CC=CC=C5)C)C',
 'C1CC(CN(C1)C2=NC=CS2)CNC(=O)C3=CC=C(C=C3)C4=CC=CS4',
 'CC1CCC2=C(C1)SC=C2C(=O)NNC(=S)NC3CCCCC3',
 'CC(C)(C1=NC(=CC=C1)N2C3=NC(=NC=C3C(=O)N2CC=C)NC4=CC=C(C=C4)N5CCN(CC5)C)O',
 'C[C@@]12CCC3(C(=CCC4[C@]3(CCC5[C@@]4(CC[C@@H]([C@]5(C)CO)OC6C(C(C(C(O6)C(=O)O)O)O)OC7C(C(C(C(O7)CO)O)O)O)C)C)[C@@H]1CC([C@H]([C@H]2OC8C(C(C(CO8)O)OC9C(C(C(C(O9)CO)O)O)O)O)O)(C)C)C']

In [87]:
tmp_msn['labels']

tensor([ True, False, False,  ..., False, False, False])

In [88]:
sum(tmp_msn['labels'])

tensor(12)

In [89]:
torch.nonzero(tmp_msn['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768, 1024, 1280, 1536, 1792, 2048, 2304, 2560, 2816])

In [90]:
len(tmp_msn['labels'])

3072

In [91]:
tmp_msn['batch_ptr']

tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256],
       dtype=torch.int32)

In [92]:
sum(tmp_msn['batch_ptr'])

tensor(3072, dtype=torch.int32)

In [93]:
len(tmp_msn['candidates'])

3072

In [94]:
tmp_msn['candidates'][5].shape

torch.Size([2048])

### Compare Shapes

In [95]:
tmp_msn.keys() == tmp_original.keys()

True

In [96]:
tmp_msn['batch_ptr'].shape == tmp_original['batch_ptr'].shape

True

In [97]:
tmp_msn['batch_ptr'].dtype == tmp_original['batch_ptr'].dtype

False

In [98]:
tmp_original['batch_ptr'].dtype

torch.int64

In [99]:
tmp_msn['batch_ptr'].dtype

torch.int32

In [100]:
sum(tmp_msn['batch_ptr']), len(tmp_msn['candidates_smiles'])

(tensor(3072, dtype=torch.int32), 3072)

In [101]:
sum(tmp_original['batch_ptr']), len(tmp_original['candidates_smiles'])

(tensor(3072), 3072)

In [102]:
tmp_msn['labels'].dtype == tmp_original['labels'].dtype

True

In [103]:
tmp_msn['candidates'].dtype == tmp_original['candidates'].dtype

True

In [104]:
tmp_msn['candidates'][0].shape == tmp_original['candidates'][0].shape

True

In [105]:
tmp_msn['candidates'][5].shape == tmp_original['candidates'][5].shape

True

In [106]:
type(tmp_msn['smiles']) == type(tmp_original['smiles'])

True

In [107]:
type(tmp_msn['smiles'][0]) == type(tmp_original['smiles'][0])

True

In [108]:
len(tmp_msn['smiles']) == len(tmp_original['smiles'])

True

In [109]:
type(tmp_msn['candidates_smiles']) == type(tmp_original['candidates_smiles'])

True

In [110]:
type(tmp_msn['candidates_smiles'][0]) == type(tmp_original['candidates_smiles'][0])

True

In [111]:
len(tmp_msn['candidates_smiles'])

3072